In [4]:
import torch
from avalanche.training.supervised import EWC
from avalanche.training.plugins import EvaluationPlugin
from avalanche.logging import TensorboardLogger
from torch.utils.tensorboard import SummaryWriter
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics, timing_metrics, forgetting_metrics,class_accuracy_metrics
from avalanche.benchmarks.classic import CORe50
from avalanche.logging import InteractiveLogger
from SimpleCNN import *
from avalanche.benchmarks.utils import as_avalanche_dataset

In [5]:
from torchvision import transforms

# Trasformazione per ridurre la risoluzione delle immagini
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Riduci la risoluzione a 64x64 (o altra risoluzione desiderata)
    transforms.ToTensor()
])

In [6]:
# Caricamento del dataset CORe50
benchmark = CORe50(scenario="nc",run=0,object_lvl=False,train_transform=transform,eval_transform=transform)
log_dir = './tb_data/experiment_SimpleCNN_EWC'

# Creazione del logger di TensorBoard per run1
tb_logger_run1 = TensorboardLogger(tb_log_dir=log_dir)

# Lista dei modelli
print("Classi Presenti : "+str(benchmark.n_classes))
model = SimpleCNN(num_classes=benchmark.n_classes)

print(f"Training model: {model.__class__.__name__}")


Loading labels...
Loading LUP...
Loading labels names...
Files already downloaded and verified
Classi Presenti : 10
Training model: SimpleCNN


In [7]:

# Plugin per la valutazione
eval_plugin = EvaluationPlugin(    
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    timing_metrics(epoch=True, epoch_running=True),
    forgetting_metrics(experience=True, stream=True),\
    class_accuracy_metrics(stream=True),\
    loggers=[tb_logger_run1])

# Definizione della strategia EWC
cl_strategy = EWC(
    model=model,
    optimizer=torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
    criterion=torch.nn.CrossEntropyLoss(),
    ewc_lambda=0.4,  # lambda per regolarizzazione EWC
    train_mb_size=32,
    train_epochs=1,
    eval_mb_size=100,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu'),
    evaluator=eval_plugin
)

for experience in benchmark.train_stream:
    print(f"Training on experience {experience.current_experience}")
    cl_strategy.train(experience)
    print("Training completed")

    print("Evaluating on test stream")
    cl_strategy.eval(benchmark.test_stream)
    print("Evaluation completed")


Training on experience 0
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 1
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 2
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 3
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 4
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 5
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 6
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 7
Training completed
Evaluating on test stream
Evaluation completed
Training on experience 8
Training completed
Evaluating on test stream
Evaluation completed
